In [ ]:
import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

In [ ]:
from wall_analysis import parse_experiments

exp_name = '2021_09_23_polar_measurement'

all_df = parse_experiments(exp_name, verbose=False)
all_df

In [ ]:
plot_spec = True
mic_idx = 1 # facing speaker
#mic_idx = 3 # away from speaker

for motors, df_motor in all_df.groupby('motors'):
    fig, ax = plt.subplots()
    for source, df in df_motor.groupby('source'):
        angles = []
        amplitudes = []

        for angle, df_angle in df.groupby('degree'):
            row = df_angle.iloc[0]
            
            angles.append(angle)

            stft = np.abs(row.stft) # times x n_mics x n_mics
            spectrogram = stft[:, mic_idx, :].T # freqs x times
            freqs = row.frequencies_matrix[0, :]

            energy = np.sum(spectrogram, axis=0)
            max_energy = np.max(energy)
            start = np.where(energy > 0.1 * max_energy)[0][0]
            cutoff = np.where(energy < 0.1 * max_energy)[0]
            cutoff = cutoff[cutoff > start][0]
            
            if plot_spec:
                fig, ax_spec = plt.subplots()
                ax_spec.pcolorfast(row.seconds, freqs, spectrogram[:-1, :-1])
                ax_spec.set_title(f'angle {angle}, source {source}')
                ax_spec.axvline(row.seconds[start], color='white')
                ax_spec.axvline(row.seconds[cutoff], color='white')
            
            amplitude = np.mean(np.max(spectrogram[:, start:cutoff], axis=0))
            amplitudes.append(amplitude)
            
        ax.plot(angles, amplitudes, label=source)
    ax.set_xlabel('angle [deg]')
    ax.set_ylabel('average maximum amplitude')
    ax.legend(title='source', loc='upper right')
    ax.set_title(f'motors {motors}')